<a href="https://colab.research.google.com/github/Yuliakoi/Speech-Emotion-Recognition/blob/main/thee_datasets_librosa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/drive/")

Drive already mounted at /drive/; to attempt to forcibly remount, call drive.mount("/drive/", force_remount=True).


In [ ]:
dirname = '/drive/My Drive/мага речевые технологии/'

In [ ]:
import pandas as pd

In [ ]:
s=pd.read_csv('/drive/My Drive/мага речевые технологии/cvs датасетов opensmile/librosa_savee.csv', header=None)

In [ ]:
b=pd.read_csv('/drive/My Drive/мага речевые технологии/cvs датасетов opensmile/librosa_berlin.csv', header=None)

In [ ]:
r=pd.read_csv('/drive/My Drive/мага речевые технологии/cvs датасетов opensmile/librosa_ravdess.csv', header=None)

In [ ]:
concated = pd.concat([s,b,r], axis=0, ignore_index=True)

In [ ]:
concated.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,a,-320.56490,110.933815,12.457850,34.670757,11.628866,-2.938115,-24.859945,-2.633692,-0.919507,-11.532021,-2.435335,-0.981383,-0.914500,-1.698188,5.144086,0.517353,-3.503025,5.504008,-0.994490,-5.816047,0.016101,0.534374,1.778036,-2.088270,-1.304948,-3.518475,-4.197404,-0.128658,-1.990220,0.656227,1.452810,1.783171,3.263025,2.756852,3.496057,5.099578,4.885279,5.633800,4.365917,3.453820
1,a,-337.16074,96.866745,4.881763,26.825062,10.724669,-2.182416,-13.945370,7.127123,-0.079203,-7.433390,1.190113,0.299027,-2.144395,0.935727,3.672377,-1.924671,3.592155,1.819140,-2.990867,0.251460,2.018039,0.099167,1.331775,-0.916692,-3.473348,-2.595617,-1.638181,0.297151,-1.132972,1.398555,2.997824,3.103378,4.924481,6.009027,6.636278,6.517982,6.425939,4.764662,3.565150,4.918954
2,a,-299.40850,120.059300,-7.834545,18.676240,14.225894,-2.376180,-20.357822,3.748768,4.587472,-11.664484,2.877178,0.469436,-4.351455,1.716199,3.823132,-3.326060,-3.007878,1.533733,-3.792394,-2.407733,0.581768,1.684659,3.417400,-0.350444,-0.505463,-2.412581,-1.405354,1.578809,-1.210608,-0.291276,1.049761,1.525852,1.987423,0.174960,-0.661589,-0.089371,-0.488557,1.521584,1.066419,-0.393784
3,a,-338.28180,123.506170,14.117958,32.017160,13.331188,2.927142,-26.489902,-4.773466,-0.012524,-10.644269,2.847523,-3.526410,-6.504219,-0.440459,5.227053,-3.274171,-6.717604,1.815684,-3.325285,-3.656561,-0.852201,-0.628887,1.528692,-3.465756,0.037132,-3.344588,-3.878498,0.846599,-3.620609,-2.362209,-2.457454,-0.681064,-0.517383,-1.376877,-0.119543,-0.691474,0.098516,1.823633,1.049620,-0.326743
4,a,-291.87260,80.724720,8.813734,24.354778,-3.727561,-2.716658,-20.333166,-4.286415,-1.203566,-7.991471,-1.519573,-4.115223,-4.809159,-1.645955,6.392706,1.171829,-0.201532,3.357184,-3.789828,-3.494981,-1.316051,-1.563732,1.218980,-2.400914,-1.070386,-1.822090,-2.669812,-0.853363,-1.285038,1.333708,4.803417,8.918399,8.608140,7.215963,6.221587,5.591549,5.749484,7.318851,6.561184,4.343619


In [ ]:
print(concated.loc[0])

0             a
1      -320.565
2       110.934
3       12.4578
4       34.6708
5       11.6289
6      -2.93812
7      -24.8599
8      -2.63369
9     -0.919507
10      -11.532
11     -2.43533
12    -0.981383
13      -0.9145
14     -1.69819
15      5.14409
16     0.517353
17     -3.50302
18      5.50401
19     -0.99449
20     -5.81605
21    0.0161008
22     0.534374
23      1.77804
24     -2.08827
25     -1.30495
26     -3.51848
27      -4.1974
28    -0.128658
29     -1.99022
30     0.656227
31      1.45281
32      1.78317
33      3.26302
34      2.75685
35      3.49606
36      5.09958
37      4.88528
38       5.6338
39      4.36592
40      3.45382
Name: 0, dtype: object


In [ ]:
for i,e in enumerate(concated.iloc[:, 0]):
  if e == 5 or e == 6 or e == 7 or e == 'W' or e == 'E' or e == 'A' or e == 'a' or e == 'd' or e == 'f':
    concated.iloc[i, 0] = "yes"
  else:
    concated.iloc[i, 0] = 'no'

In [ ]:
concated.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,yes,-320.56490,110.933815,12.457850,34.670757,11.628866,-2.938115,-24.859945,-2.633692,-0.919507,-11.532021,-2.435335,-0.981383,-0.914500,-1.698188,5.144086,0.517353,-3.503025,5.504008,-0.994490,-5.816047,0.016101,0.534374,1.778036,-2.088270,-1.304948,-3.518475,-4.197404,-0.128658,-1.990220,0.656227,1.452810,1.783171,3.263025,2.756852,3.496057,5.099578,4.885279,5.633800,4.365917,3.453820
1,yes,-337.16074,96.866745,4.881763,26.825062,10.724669,-2.182416,-13.945370,7.127123,-0.079203,-7.433390,1.190113,0.299027,-2.144395,0.935727,3.672377,-1.924671,3.592155,1.819140,-2.990867,0.251460,2.018039,0.099167,1.331775,-0.916692,-3.473348,-2.595617,-1.638181,0.297151,-1.132972,1.398555,2.997824,3.103378,4.924481,6.009027,6.636278,6.517982,6.425939,4.764662,3.565150,4.918954
2,yes,-299.40850,120.059300,-7.834545,18.676240,14.225894,-2.376180,-20.357822,3.748768,4.587472,-11.664484,2.877178,0.469436,-4.351455,1.716199,3.823132,-3.326060,-3.007878,1.533733,-3.792394,-2.407733,0.581768,1.684659,3.417400,-0.350444,-0.505463,-2.412581,-1.405354,1.578809,-1.210608,-0.291276,1.049761,1.525852,1.987423,0.174960,-0.661589,-0.089371,-0.488557,1.521584,1.066419,-0.393784
3,yes,-338.28180,123.506170,14.117958,32.017160,13.331188,2.927142,-26.489902,-4.773466,-0.012524,-10.644269,2.847523,-3.526410,-6.504219,-0.440459,5.227053,-3.274171,-6.717604,1.815684,-3.325285,-3.656561,-0.852201,-0.628887,1.528692,-3.465756,0.037132,-3.344588,-3.878498,0.846599,-3.620609,-2.362209,-2.457454,-0.681064,-0.517383,-1.376877,-0.119543,-0.691474,0.098516,1.823633,1.049620,-0.326743
4,yes,-291.87260,80.724720,8.813734,24.354778,-3.727561,-2.716658,-20.333166,-4.286415,-1.203566,-7.991471,-1.519573,-4.115223,-4.809159,-1.645955,6.392706,1.171829,-0.201532,3.357184,-3.789828,-3.494981,-1.316051,-1.563732,1.218980,-2.400914,-1.070386,-1.822090,-2.669812,-0.853363,-1.285038,1.333708,4.803417,8.918399,8.608140,7.215963,6.221587,5.591549,5.749484,7.318851,6.561184,4.343619


In [ ]:
concated.to_csv('/drive/My Drive/мага речевые технологии/cvs датасетов opensmile/librosa_общий_бинарный.csv', index=False, header=False)

разделение на выборки

In [ ]:
X_ready = concated.drop(([0]), axis=1)
Y_ready = concated[0]

In [ ]:
#изменить формат 
#for i,b in enumerate(Y_ready):
    #Y_ready[i]=b.astype('int')

IndentationError: ignored

**Базовые модели**

In [ ]:
#импорт всех моделей
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier, PassiveAggressiveClassifier, Perceptron

from sklearn.svm import SVC

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [ ]:
models = {
  'log': LogisticRegression(),
  'sgd': SGDClassifier(),
  'ridge': RidgeClassifier(),
  'pac': PassiveAggressiveClassifier(),
  'perc': Perceptron(),

  'linear_svc': SVC(kernel='linear'),
  'rbf_svc': SVC(kernel='rbf'),
  'poly_svc': SVC(kernel='poly'),

  'mlp': MLPClassifier(),
  'knn': KNeighborsClassifier(),
  'gpc': GaussianProcessClassifier(),
  'qda': QuadraticDiscriminantAnalysis(),

  'gnb': GaussianNB(),
  'bnb': BernoulliNB(),

  'tree': DecisionTreeClassifier(),

  'forest': RandomForestClassifier(),
  'ada': AdaBoostClassifier(),
  'extra': ExtraTreesClassifier(),
  'gboost': GradientBoostingClassifier(),
}
print(len(models))

19




```
# Выбран кодовый формат
```

**Модели с пайплайном**

In [ ]:
import time

import numpy as np
import pandas as pd

import os

In [ ]:
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def test_model(model, x, y):  
  x_train, x_test, y_train, y_test = train_test_split(X_ready, Y_ready, test_size=0.3, stratify = Y_ready)
  
  pipe = Pipeline([
                 ('scaler', StandardScaler()), 
                 ('cls', models[model])
                 ])
  start_time = time.time()
  pipe.fit(x_train, y_train)
  time_fit="%s sec" % (time.time() - start_time)
  #fit для тестовой и тренировочной выборки
  y_test_pred =pipe.predict(x_test)
  y_train_pred =pipe.predict(x_train)
  #accuracy для предсказанных значений по test и train
  acc_test = accuracy_score(y_test, y_test_pred)
  acc_train = accuracy_score(y_train, y_train_pred)
  prec_test = precision_score(y_test, y_test_pred, average=None).mean()
  recall_test = recall_score(y_test, y_test_pred, average=None).mean()
  #precision & recall для test
  #prec_test = precision_score(y_test, y_test_pred)
  #recall_test = recall_score(y_test, y_test_pred, average='macro')


  

  
  scores = cross_val_score(pipe, x, y, cv=5).mean()

  return {
      "MODEL": model,
      "SCORE":scores,
      "ACCURACY TRAIN":acc_train,
      "ACCURACY TEST":acc_test,
      "PRECISION TEST":prec_test,
      "RECALL TEST":recall_test,
      "TIME FOR FIT":time_fit 
      }

In [ ]:
test_model('forest', X_ready, Y_ready)

{'ACCURACY TEST': 0.7951153324287653,
 'ACCURACY TRAIN': 1.0,
 'MODEL': 'forest',
 'PRECISION TEST': 0.8017363240002725,
 'RECALL TEST': 0.7702784134248666,
 'SCORE': 0.6496945010183299,
 'TIME FOR FIT': '0.8668115139007568 sec'}

In [ ]:
for name, model in models.items():
  res = test_model(name, X_ready, Y_ready)
  #table=pd.DataFrame.from_dict(res)
  print(name, res)

log {'MODEL': 'log', 'SCORE': 0.6093686354378819, 'ACCURACY TRAIN': 0.6949941792782305, 'ACCURACY TEST': 0.6906377204884667, 'PRECISION TEST': 0.6797542646227513, 'RECALL TEST': 0.6649351639969489, 'TIME FOR FIT': '0.050115346908569336 sec'}
sgd {'MODEL': 'sgd', 'SCORE': 0.5780040733197557, 'ACCURACY TRAIN': 0.6612339930151339, 'ACCURACY TEST': 0.6390773405698779, 'PRECISION TEST': 0.627536450857445, 'RECALL TEST': 0.6287719298245614, 'TIME FOR FIT': '0.03157758712768555 sec'}
ridge {'MODEL': 'ridge', 'SCORE': 0.6105906313645622, 'ACCURACY TRAIN': 0.710710128055879, 'ACCURACY TEST': 0.6784260515603799, 'PRECISION TEST': 0.6664556962025316, 'RECALL TEST': 0.6504576659038901, 'TIME FOR FIT': '0.018865346908569336 sec'}
pac {'MODEL': 'pac', 'SCORE': 0.5792260692464358, 'ACCURACY TRAIN': 0.6385331781140862, 'ACCURACY TEST': 0.6132971506105834, 'PRECISION TEST': 0.6037714712471994, 'RECALL TEST': 0.6059877955758963, 'TIME FOR FIT': '0.010942220687866211 sec'}
perc {'MODEL': 'perc', 'SCORE':

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


mlp {'MODEL': 'mlp', 'SCORE': 0.6875763747454174, 'ACCURACY TRAIN': 0.9935972060535506, 'ACCURACY TEST': 0.8073270013568521, 'PRECISION TEST': 0.8034478409610444, 'RECALL TEST': 0.7936384439359268, 'TIME FOR FIT': '2.3463871479034424 sec'}
knn {'MODEL': 'knn', 'SCORE': 0.5861507128309572, 'ACCURACY TRAIN': 0.8818393480791619, 'ACCURACY TEST': 0.7842605156037992, 'PRECISION TEST': 0.7786147186147185, 'RECALL TEST': 0.7700076277650649, 'TIME FOR FIT': '0.01318669319152832 sec'}
gpc {'MODEL': 'gpc', 'SCORE': 0.6069246435845214, 'ACCURACY TRAIN': 0.9994179278230501, 'ACCURACY TEST': 0.8181818181818182, 'PRECISION TEST': 0.8202181249082995, 'RECALL TEST': 0.7996567505720824, 'TIME FOR FIT': '2.28342866897583 sec'}
qda {'MODEL': 'qda', 'SCORE': 0.5942973523421589, 'ACCURACY TRAIN': 0.7712456344586729, 'ACCURACY TEST': 0.7055630936227951, 'PRECISION TEST': 0.6950481341962071, 'RECALL TEST': 0.684836003051106, 'TIME FOR FIT': '0.01861572265625 sec'}
gnb {'MODEL': 'gnb', 'SCORE': 0.608146639511

In [ ]:
table = pd.DataFrame()
for name, _ in models.items():
  res = pd.DataFrame(test_model(name, X_ready, Y_ready), index=[name, ])
  table = pd.concat([table, res], axis=0)
table.head()

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


,MODEL,SCORE,ACCURACY TRAIN,ACCURACY TEST,PRECISION TEST,RECALL TEST,TIME FOR FIT
log,log,0.609369,0.691502,0.700136,0.690368,0.675034,0.032752037048339844 sec
sgd,sgd,0.595519,0.674040,0.658073,0.645028,0.643745,0.03747963905334473 sec
ridge,ridge,0.610591,0.704889,0.687924,0.676535,0.662647,0.018474578857421875 sec
pac,pac,0.531976,0.627474,0.618725,0.598656,0.590709,0.010423898696899414 sec
perc,perc,0.613035,0.614668,0.595658,0.581899,0.582231,0.010302543640136719 sec


In [ ]:
table.drop('MODEL', axis=1, inplace=True)

In [ ]:
table

,SCORE,ACCURACY TRAIN,ACCURACY TEST,PRECISION TEST,RECALL TEST,TIME FOR FIT
log,0.609369,0.691502,0.700136,0.690368,0.675034,0.032752037048339844 sec
sgd,0.595519,0.674040,0.658073,0.645028,0.643745,0.03747963905334473 sec
ridge,0.610591,0.704889,0.687924,0.676535,0.662647,0.018474578857421875 sec
pac,0.531976,0.627474,0.618725,0.598656,0.590709,0.010423898696899414 sec
perc,0.613035,0.614668,0.595658,0.581899,0.582231,0.010302543640136719 sec
linear_svc,0.601629,0.710710,0.685210,0.674761,0.656178,0.37978100776672363 sec
rbf_svc,0.652138,0.869616,0.769335,0.778582,0.739134,0.20949125289916992 sec
poly_svc,0.647658,0.814319,0.727273,0.768367,0.677540,0.15039920806884766 sec
mlp,0.682688,0.987776,0.838535,0.838625,0.824134,2.398763656616211 sec
knn,0.586151,0.871362,0.763908,0.757207,0.748143,0.012981414794921875 sec


In [ ]:
table.to_csv('/drive/My Drive/мага речевые технологии/cvs датасетов opensmile/librosa_бинарный_метрики.csv')